In [28]:
!pip install pynvml 

Looking in indexes: http://jfrog.fkinternal.com/artifactory/api/pypi/python_virtual/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 555.1 kB/s eta 0:00:00 0:00:01


In [1]:
import json
import time
import torch
from transformers import TextClassificationPipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForMaskedLM
import numpy as np
import pandas as pd
from h3 import h3
from geopy.distance import geodesic as vincenty
from tqdm import tqdm
from datasets import load_dataset

/home/uddeshya.singh/miniconda3/envs/uddeshya_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoTokenizer
from transformers import DataCollatorForLanguageModeling
from transformers import AutoConfig, RobertaForMaskedLM
from transformers import Trainer, TrainingArguments
from transformers import pipeline
import time
from transformers import AutoModelForSequenceClassification, DataCollatorWithPadding
import torch

In [3]:
def load_finetuned_model(checkpoint_path, base_tokenizer_path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = AutoModelForMaskedLM.from_pretrained(checkpoint_path)
    tokenizer = AutoTokenizer.from_pretrained(base_tokenizer_path)
    model = model.to(device)
    return model, tokenizer, device

In [4]:
checkpoint_path =  '/home/uddeshya.singh/Experiments/punjab_roberta_model_without_city_05/checkpoint_epoch2'
model, tokenizer , device = load_finetuned_model(checkpoint_path, checkpoint_path)

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at /home/uddeshya.singh/Experiments/punjab_roberta_model_without_city_05/checkpoint_epoch2 and are newly initialized: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
address_data_path = '/home/uddeshya.singh/DataSet_triplet/classification_dataset/PUNJAB_with_grid_ids_train.csv'
print('starting now ', time.strftime('%X %x %Z'))
address_data_train = load_dataset("csv", data_files=address_data_path)
print('completed now ', time.strftime('%X %x %Z'))
address_data_train

starting now  10:17:31 07/03/24 IST
completed now  10:18:52 07/03/24 IST


DatasetDict({
    train: Dataset({
        features: ['addr_hash', 'cleaned_address', 'lookup_lat', 'lookup_lng', 'grid_id'],
        num_rows: 6913138
    })
})

In [6]:
print('starting now ', time.strftime('%X %x %Z'))
address_data_train["train"] = address_data_train["train"].class_encode_column("grid_id")
print('starting now ', time.strftime('%X %x %Z'))
address_data_train = address_data_train.remove_columns(['addr_hash', 'lookup_lat', 'lookup_lng'])
print('starting now ', time.strftime('%X %x %Z'))
address_data_train = address_data_train.rename_column( 'grid_id', 'labels')
print('starting now ', time.strftime('%X %x %Z'))



starting now  10:18:52 07/03/24 IST
starting now  10:18:56 07/03/24 IST
starting now  10:18:57 07/03/24 IST
starting now  10:18:58 07/03/24 IST


In [7]:
class_label_feature = address_data_train['train'].features["labels"]
class_label_feature.num_classes

161419

In [8]:
num_labels = class_label_feature.num_classes
class_names = class_label_feature.names
print(f"number of labels: {num_labels}")

id2label = {i: label for i, label in enumerate(class_names)}
id2label[len(class_names)] = 'None'

label2id = {label:id for (id,label) in id2label.items()}
print(len(id2label), len(label2id))

number of labels: 161419
161420 161420


In [9]:
import json
save_dir = '/home/uddeshya.singh/DataSet_triplet/classification_dataset/ID_to_L'
save_id2label = save_dir +'/id2label.json'
with open(save_id2label, 'w') as f:
    json.dump(id2label, f)

save_label2id = save_dir +'/label2id.json'
with open(save_label2id, 'w') as f:
    json.dump(label2id, f)

In [10]:
pretrained_tokenizer = tokenizer

In [11]:
txt = "kadubeesanhalli salarpuria touch stone building"
tokens = pretrained_tokenizer(txt)['input_ids']
print(tokens)
converted = pretrained_tokenizer.convert_ids_to_tokens(tokens)
print(converted)

[0, 2639, 356, 5383, 10881, 13242, 16162, 6028, 556, 2]
['<s>', 'kad', 'ub', 'ees', 'anhalli', 'Ġsalarpuria', 'Ġtouch', 'Ġstone', 'Ġbuilding', '</s>']


In [12]:
def tokenize(batch):
    tokenized_batch = pretrained_tokenizer(batch["cleaned_address"], padding=True, truncation=True, max_length=100)#, is_split_into_words=True)
    return tokenized_batch

In [13]:
import pandas as pd
from tqdm import tqdm

# identify the issue rows
print('starting now ', time.strftime('%X %x %Z'))

none_ind_list = []
for i, a in tqdm(enumerate(address_data_train['train']['cleaned_address'])):
    if a is None or pd.isnull(a) or a=='':
        # print("a is None")
        none_ind_list.append(i)
        
print(len(none_ind_list), none_ind_list)

print('starting now ', time.strftime('%X %x %Z'))

# Exclude specific rows from the dataset

new_address_data_train = address_data_train['train'].select(
    (
        i for i in range(len(address_data_train['train'])) 
        if i not in set(none_ind_list)
    )
)

print('starting now ', time.strftime('%X %x %Z'))
new_address_data_train

starting now  10:19:41 07/03/24 IST


6913138it [00:04, 1494224.32it/s]
Parameter 'indices'=<generator object <genexpr> at 0x7fd2a02d8200> of the transform datasets.arrow_dataset.Dataset.select couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


20 [1766592, 2158403, 2158404, 2216726, 2989370, 3178074, 3811428, 3979659, 4076605, 4429631, 4560339, 4608657, 4608658, 4996408, 5176882, 5341938, 5720808, 6000192, 6241250, 6463379]
starting now  10:19:52 07/03/24 IST
starting now  10:19:59 07/03/24 IST


Dataset({
    features: ['cleaned_address', 'labels'],
    num_rows: 6913118
})

In [14]:
print('starting now ', time.strftime('%X %x %Z'))
# train_dataset = address_data_train['train'].map(tokenize, batched=True, batch_size=10) #len(address_data_train['train']))
train_dataset = new_address_data_train.map(tokenize, batched=True, batch_size=100000, num_proc=7) #en(new_address_data_train))
train_dataset = train_dataset.remove_columns(['cleaned_address'])
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
print('starting now ', time.strftime('%X %x %Z'))
train_dataset

starting now  10:20:04 07/03/24 IST


Map (num_proc=7): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6913118/6913118 [02:15<00:00, 51035.35 examples/s]


starting now  10:22:38 07/03/24 IST


Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 6913118
})

In [15]:
from pynvml import *

def get_gpu_device_memory(device_index):
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(device_index)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU Device {device_index} and memory occupied: {info.used//1024**2} MB.")

    memory = info.used//1024**2

def print_gpu_utilization():

    for d in [0,1,2,3]:
        get_gpu_device_memory(d)
    
    # nvmlInit()
    # handle = nvmlDeviceGetHandleByIndex(0)
    # info = nvmlDeviceGetMemoryInfo(handle)
    # print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

In [16]:
print_gpu_utilization()


GPU Device 0 and memory occupied: 895 MB.
GPU Device 1 and memory occupied: 445 MB.
GPU Device 2 and memory occupied: 445 MB.
GPU Device 3 and memory occupied: 445 MB.


In [17]:
data_collator = DataCollatorWithPadding(tokenizer=pretrained_tokenizer)

In [24]:
print('starting now ', time.strftime('%X %x %Z'))

model_checkpoint = '/home/uddeshya.singh/Experiments/punjab_roberta_model_without_city_05/checkpoint_epoch2'

with torch.cuda.device(2):
    model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=len(label2id))#.to("cuda")

print('starting now ', time.strftime('%X %x %Z'))
print_gpu_utilization()

starting now  10:29:20 07/03/24 IST


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /home/uddeshya.singh/Experiments/punjab_roberta_model_without_city_05/checkpoint_epoch2 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


starting now  10:29:23 07/03/24 IST
GPU Device 0 and memory occupied: 4671 MB.
GPU Device 1 and memory occupied: 2411 MB.
GPU Device 2 and memory occupied: 2411 MB.
GPU Device 3 and memory occupied: 2411 MB.


In [25]:
NO_OF_EPOCHS = 20
fine_tuned_lm_trained_out_dir = '/home/uddeshya.singh/Experiments/Triplet_models_classification'
print('starting now ', time.strftime('%X %x %Z'))
training_args = TrainingArguments(
 output_dir=fine_tuned_lm_trained_out_dir,
 overwrite_output_dir=True,
 num_train_epochs=NO_OF_EPOCHS,
 per_device_train_batch_size=4,
 logging_steps = 10000,
 save_steps=20000,
 save_total_limit=5,

    
#     gradient_accumulation_steps=4,
#     gradient_checkpointing=True,
#     gradient_checkpointing_kwargs={'use_reentrant':False},
#     fp16=True
    
    
#  evaluation_strategy="steps",
#  load_best_model_at_end=True
)

starting now  10:29:25 07/03/24 IST


In [26]:
trainer = Trainer(
 model=model,
 args=training_args,
 data_collator=data_collator,
 tokenizer=pretrained_tokenizer,
 train_dataset=train_dataset
)
print('starting now ', time.strftime('%X %x %Z'))
print_gpu_utilization()

RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [22]:
print('starting now ', time.strftime('%X %x %Z'))
start_time = time.time()
result = trainer.train()
end_time = time.time()
print('completed now ', time.strftime('%X %x %Z'))

starting now  10:28:10 07/03/24 IST


RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
trainer.save_model(fine_tuned_lm_trained_save_dir)
trainer.save_state()

In [43]:
import json
import time
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForSequenceClassification, DataCollatorWithPadding, Trainer, TrainingArguments
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm

# Load fine-tuned model and tokenizer
def load_finetuned_model(checkpoint_path, base_tokenizer_path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = AutoModelForMaskedLM.from_pretrained(checkpoint_path)
    tokenizer = AutoTokenizer.from_pretrained(base_tokenizer_path)
    model = model.to(device)
    return model, tokenizer, device

checkpoint_path = '/home/uddeshya.singh/Experiments/punjab_roberta_model_without_city_05/checkpoint_epoch2'
model, tokenizer, device = load_finetuned_model(checkpoint_path, checkpoint_path)

# Load dataset
address_data_path = '/home/uddeshya.singh/DataSet_triplet/classification_dataset/PUNJAB_with_grid_ids_train.csv'
print('Loading dataset... ', time.strftime('%X %x %Z'))
address_data_train = load_dataset("csv", data_files=address_data_path)
print('Dataset loaded. ', time.strftime('%X %x %Z'))

# Preprocess dataset
print('Preprocessing dataset... ', time.strftime('%X %x %Z'))
address_data_train["train"] = address_data_train["train"].class_encode_column("grid_id")
address_data_train = address_data_train.remove_columns(['addr_hash', 'lookup_lat', 'lookup_lng'])
address_data_train = address_data_train.rename_column('grid_id', 'labels')

class_label_feature = address_data_train['train'].features["labels"]
num_labels = class_label_feature.num_classes
class_names = class_label_feature.names
print(f"Number of labels: {num_labels}")

id2label = {i: label for i, label in enumerate(class_names)}
id2label[len(class_names)] = 'None'
label2id = {label: id for (id, label) in id2label.items()}

save_dir = '/home/uddeshya.singh/DataSet_triplet/classification_dataset/ID_to_L'
save_id2label = save_dir + '/id2label.json'
with open(save_id2label, 'w') as f:
    json.dump(id2label, f)

save_label2id = save_dir + '/label2id.json'
with open(save_label2id, 'w') as f:
    json.dump(label2id, f)

# Tokenizer example
pretrained_tokenizer = tokenizer
txt = "kadubeesanhalli salarpuria touch stone building"
tokens = pretrained_tokenizer(txt)['input_ids']
print(tokens)
converted = pretrained_tokenizer.convert_ids_to_tokens(tokens)
print(converted)

# Tokenize function
def tokenize(batch):
    tokenized_batch = pretrained_tokenizer(batch["cleaned_address"], padding=True, truncation=True, max_length=100)
    return tokenized_batch

# Identify issue rows
print('Identifying issue rows... ', time.strftime('%X %x %Z'))
none_ind_list = [i for i, a in tqdm(enumerate(address_data_train['train']['cleaned_address'])) if a is None or pd.isnull(a) or a == '']
print(f"Found {len(none_ind_list)} issue rows.")

# Exclude specific rows from the dataset
new_address_data_train = address_data_train['train'].select(
    i for i in range(len(address_data_train['train'])) if i not in set(none_ind_list)
)

# Tokenize dataset
print('Tokenizing dataset... ', time.strftime('%X %x %Z'))
train_dataset = new_address_data_train.map(tokenize, batched=True, batch_size=100000, num_proc=7)
train_dataset = train_dataset.remove_columns(['cleaned_address'])
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
print('Dataset tokenized. ', time.strftime('%X %x %Z'))

data_collator = DataCollatorWithPadding(tokenizer=pretrained_tokenizer)

# Load model for sequence classification
model_checkpoint = '/home/uddeshya.singh/Experiments/punjab_roberta_model_without_city_05/checkpoint_epoch2'
with torch.cuda.device(2):
    model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=len(label2id))

# Training setup
NO_OF_EPOCHS = 20
fine_tuned_lm_trained_out_dir = '/home/uddeshya.singh/Experiments/Triplet_models_classification'
training_args = TrainingArguments(
    output_dir=fine_tuned_lm_trained_out_dir,
    overwrite_output_dir=True,
    num_train_epochs=NO_OF_EPOCHS,
    per_device_train_batch_size=4,
    logging_steps=10000,
    save_steps=20000,
    save_total_limit=5
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    tokenizer=pretrained_tokenizer,
    train_dataset=train_dataset
)

# Train model
print('Training model... ', time.strftime('%X %x %Z'))
start_time = time.time()
result = trainer.train()
end_time = time.time()
print('Training completed. ', time.strftime('%X %x %Z'))

trainer.save_model(fine_tuned_lm_trained_out_dir)
trainer.save_state()

print(f"Total training time: {end_time - start_time} seconds")


Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at /home/uddeshya.singh/Experiments/punjab_roberta_model_without_city_05/checkpoint_epoch2 and are newly initialized: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading dataset...  21:13:28 07/01/24 IST
Dataset loaded.  21:14:48 07/01/24 IST
Preprocessing dataset...  21:14:48 07/01/24 IST
Number of labels: 161419
[0, 2639, 356, 5383, 10881, 13242, 16162, 6028, 556, 2]
['<s>', 'kad', 'ub', 'ees', 'anhalli', 'Ġsalarpuria', 'Ġtouch', 'Ġstone', 'Ġbuilding', '</s>']
Identifying issue rows...  21:14:55 07/01/24 IST


6913138it [00:04, 1720240.07it/s]


Found 20 issue rows.
Tokenizing dataset...  21:15:11 07/01/24 IST


Map (num_proc=7): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6913118/6913118 [02:10<00:00, 53005.88 examples/s]


Dataset tokenized.  21:17:40 07/01/24 IST


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /home/uddeshya.singh/Experiments/punjab_roberta_model_without_city_05/checkpoint_epoch2 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training model...  21:17:43 07/01/24 IST


/home/uddeshya.singh/miniconda3/envs/uddeshya_env/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


KeyboardInterrupt: 